# Main

#### Setup

In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# Create the connection with Google Cloud
def create_connection():
        # Set the credentials and create the connection
        key_path = "../usp-mba-dsa-tcc-4277103d9155.json"

        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/bigquery"]
        )

        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id,
        )
        return client

client = create_connection()

In [3]:
# Perform a query.
QUERY = ("""
    SELECT DISTINCT offer_id, LOWER(Descricao) Descricao
    FROM `usp-mba-dsa-tcc.ecommerce_offers.vw_dim_offers`
""")
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

offers = rows.to_dataframe()

In [4]:
offers.head(2)

,offer_id,Descricao
0,272b754da4d4be1ab0871ea02023adeebbad9c906363b6...,ração n&amp;d prime cães adultos maxi frango 1...
1,9eab4767c70bee1fd0f2369bda3af61fc4d0b8293198dd...,ração granplus menu carne e arroz para cães fi...


In [5]:
# Carrega a lista de sabores
lista_sabores = list(set(pd.read_csv('sabores.csv')['words']))
len(lista_sabores)

109

#### Transformações

In [6]:
from nltk.tokenize import word_tokenize
offers["Sabores"] = offers['Descricao'].apply(word_tokenize) 

In [7]:
# Filtrar sabores da lista
def filtrar_sabores(lista):
    lista_filtrada = [sabor.capitalize() for sabor in lista if sabor in lista_sabores]
    return sorted(lista_filtrada)

offers['Sabores'] = offers['Sabores'].apply(filtrar_sabores)

In [8]:
# Transformar listas de sabores em string
def transformar_lista_para_string(lista):
    # Verifica se a lista está vazia
    if not lista:
        return None
    
    # Se houver apenas um elemento, retorna esse elemento como uma string
    if len(lista) == 1:
        return lista[0]
    
    # Se houver mais de um elemento, formata a lista como uma string com "e" entre os dois últimos elementos
    return ', '.join(lista[:-1]) + ' e ' + lista[-1]

offers['Sabores'] = offers['Sabores'].apply(transformar_lista_para_string)

In [9]:
# Drop column Descricao
offers.drop('Descricao', axis=1, inplace=True)
offers.head(2)

,offer_id,Sabores
0,272b754da4d4be1ab0871ea02023adeebbad9c906363b6...,Frango
1,9eab4767c70bee1fd0f2369bda3af61fc4d0b8293198dd...,Arroz e Carne


#### Upload

In [10]:
# Salva dataframe como tabela no BQ (substituindo)
project_id = client.project

offers.to_gbq(
    project_id=project_id,
    destination_table='ecommerce_offers.tb_sabores',
    if_exists='replace'
)

100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]


# Outros

#### Similarity

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_lg")

In [ ]:
for word in offers["Sabores"][0]:
    sim_score = nlp(word).similarity(nlp("carne"))
    print(word, sim_score)

In [ ]:
descricoes = ' '.join(offers['Descricao'])
words = list(set(word_tokenize(descricoes))) # Lista distinta de tokens
print(len(words))

#### All words

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('portuguese'))

In [ ]:
import re

def remove_stopwords_and_numbers(words):
    filtered_tokens = []
    for token in words:
        # Verifique se o token não é uma stop word e não é um número
        if token.lower() not in stop_words and not re.match(r'^\d+(?:\.\d+)?$', token):
            filtered_tokens.append(token)
    return filtered_tokens

tokens_sem_stopwords_e_numeros = remove_stopwords_and_numbers(words)
print(len(tokens_sem_stopwords_e_numeros))
tokens_sem_stopwords_e_numeros

In [ ]:
df = pd.DataFrame(tokens_sem_stopwords_e_numeros)
df.to_csv('xaxaxa.csv', index=False)

In [ ]:
offers["Sabores"][1]

#### Teste

In [ ]:
sabores = []
doc = nlp(offers["Descricao"][0])

# Itere pelas entidades nomeadas no texto
for entidade in doc.ents:
    if entidade.label_ == "sabor":  # Verifique se a entidade é um sabor
        sabores.append(entidade.text)

In [ ]:
sabores

https://docs.google.com/spreadsheets/d/1q5zumCqCpdy4qAMwD1ATpB4FVmgSrmJ9Pliyr-Y_D6w/edit#gid=853736561
1. SALVAR UMA LISTA DE SABORES E APLICAR E JÁ ERA
2. MONITORAR PRODUTOS SEM SABORES PARA VER O QUE DEU

Resolver problemas:
Frutos = Frutos do Mar
cereias x cereais